Coursera Capstone Final

In [20]:
import numpy as np
import pandas as pd
import requests
import geocoder
import pickle
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import json
from pandas.io.json import json_normalize 
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

Load Dataframe from Part 2

In [21]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [22]:
raw_random_wikipedia_page = requests.get(wikipedia_link)
page = raw_random_wikipedia_page.text
page

u'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of postal codes of Canada: M - Wikipedia</title>\n<script>document.documentElement.className = document.documentElement.className.replace( /(^|\\s)client-nojs(\\s|$)/, "$1client-js$2" );</script>\n<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":862527922,"wgRevisionId":862527922,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat

In [23]:
def parse_html(table):
    n_columns = 0
    n_rows=0
    column_names = []

    # Find number of rows and columns
    # we also find the column titles if we can
    for row in table.find_all('tr'):

        # Determine the number of rows in the table
        td_tags = row.find_all('td')
        if len(td_tags) > 0:
            n_rows+=1
            if n_columns == 0:
                # Set the number of columns for our table
                n_columns = len(td_tags)

        # Handle column names if we find them
        th_tags = row.find_all('th') 
        if len(th_tags) > 0 and len(column_names) == 0:
            for th in th_tags:
                column_names.append(th.get_text())

    # Safeguard on Column Titles
    if len(column_names) > 0 and len(column_names) != n_columns:
        raise Exception("Column titles do not match the number of columns")

    columns = column_names if len(column_names) > 0 else range(0,n_columns)
    df = pd.DataFrame(columns = columns,
                      index= range(0,n_rows))
    row_marker = 0
    for row in table.find_all('tr'):
        column_marker = 0
        columns = row.find_all('td')
        #pr
        for column in columns:
            df.iat[row_marker,column_marker] = column.get_text()
            column_marker += 1
        if len(columns) > 0:
            row_marker += 1

    # Convert to float if possible
    for col in df:
        try:
            df[col] = df[col].astype(float)
        except ValueError:
            pass

    return df

In [24]:
soup = BeautifulSoup(page, 'lxml') # Parse the HTML as a string

table = soup.find_all('table')[0] # Grab the first table
df =parse_html(table)
df['Neighbourhood\n'] = df['Neighbourhood\n'].str.replace('\n','')
df=df.rename(columns = {'Neighbourhood\n':'Neighbourhood'})
df =df[df['Borough']!= 'Not assigned']

In [25]:
postcode = []
borough= []
neighbor = []
groups = df.groupby('Postcode')
for index,group in groups:
    postcode.append(index)
    borough.append(group['Borough'].max())
    if group['Neighbourhood'].max()== 'Not assigned':
        neighbor.append(group['Borough'].max())
    else:
        neighbor.append(", ".join(list(group['Neighbourhood'])).replace('[','').replace(']',''))

In [26]:
df2 = pd.DataFrame(
    {'Postcode': postcode,
     'Borough': borough,
     'Neighbourhood': neighbor
    })

In [27]:
cols = df2.columns.tolist()
cols = cols[-1:] + cols[:-1]
df =df2[cols]
df.shape

(103, 3)

In [28]:
geolocator = Nominatim(user_agent="Toronto")
def get_lat_lng(postal_code):
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        #g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        #lat_lng_coords = g.latlng
        lat_lng_coords = geolocator.geocode('{}, Toronto, Ontario'.format(postal_code))
        
    return lat_lng_coords.latitude,lat_lng_coords.longitude

In [29]:
bor = df.Borough.tolist()
bor= np.unique(bor)
len(bor)

11

In [30]:
longitudes = []
latitude = []

for ind in bor:
    lat,lng = get_lat_lng(ind.strip())
    longitudes.append(lng)
    latitude.append(lat)
    print ind, lat,lng

Central Toronto 43.6449033 -79.3818364
Downtown Toronto 43.655115 -79.380219
East Toronto 43.6449033 -79.3818364
East York 43.6913391 -79.3278212
Etobicoke 43.6435559 -79.5656326
Mississauga 43.6679336 -79.5874178
North York 43.7709163 -79.4124102
Queen's Park 43.6599803 -79.3903686
Scarborough 43.773077 -79.257774
West Toronto 43.6449033 -79.3818364
York 43.6896191 -79.479188


In [31]:
df2 = pd.DataFrame(
    {'Borough': bor,
     'Latitude': latitude,
     'Longitude':longitudes
    })

In [32]:
df2

,Borough,Latitude,Longitude
0,Central Toronto,43.644903,-79.381836
1,Downtown Toronto,43.655115,-79.380219
2,East Toronto,43.644903,-79.381836
3,East York,43.691339,-79.327821
4,Etobicoke,43.643556,-79.565633
5,Mississauga,43.667934,-79.587418
6,North York,43.770916,-79.412410
7,Queen's Park,43.659980,-79.390369
8,Scarborough,43.773077,-79.257774
9,West Toronto,43.644903,-79.381836


In [33]:

lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
    lat_lng_coords = geolocator.geocode('{}, Toronto, Ontario')
    #g = geocoder.google('{}, Toronto, Ontario')
    #lat_lng_coords = g.latlng

tor_lat =lat_lng_coords.latitude
tor_long =lat_lng_coords.longitude

#### Use geocoder library to get the latitude and longitude values of Toronto City.

#### Create a map of Toronto City with neighborhoods superimposed on top.

In [34]:
map_toronto = folium.Map(location=[tor_lat, tor_long], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(df2['Latitude'], df2['Longitude'], df['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [35]:
CLIENT_ID = 'SCH5UTBS50JUZVK00TIXXAS2RD5J1CMZOFQSBUDICG35TX0Y' # your Foursquare ID
CLIENT_SECRET = 'WNGTUZCA4RPXDZGR3DNWIUWNNP3XLHRVZ2V3DCEHKS41B3Q4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SCH5UTBS50JUZVK00TIXXAS2RD5J1CMZOFQSBUDICG35TX0Y
CLIENT_SECRET:WNGTUZCA4RPXDZGR3DNWIUWNNP3XLHRVZ2V3DCEHKS41B3Q4


In [36]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
'''
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
'''

"\nurl = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(\n    CLIENT_ID, \n    CLIENT_SECRET, \n    VERSION, \n    neighborhood_latitude, \n    neighborhood_longitude, \n    radius, \n    LIMIT)\nurl # display URL\n"

In [37]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
borough_venues = getNearbyVenues(names=df2['Borough'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude']
                                  )

Central Toronto
Downtown Toronto
East Toronto
East York
Etobicoke
Mississauga
North York
Queen's Park
Scarborough
West Toronto
York


In [40]:
east_data = df[df['Borough']=='East Toronto'].reset_index(drop=True)
east_data.head()

,Postcode,Borough,Neighbourhood
0,M4E,East Toronto,The Beaches
1,M4K,East Toronto,"The Danforth West, Riverdale"
2,M4L,East Toronto,"The Beaches West, India Bazaar"
3,M4M,East Toronto,Studio District
4,M7Y,East Toronto,Business reply mail Processing Centre969 Eastern


Let's get the geographical coordinates of East Toronto.

In [72]:
print(borough_venues.shape)
borough_venues

(585, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Central Toronto,43.644903,-79.381836,Union Pearson Express,43.644362,-79.383199,Train Station
1,Central Toronto,43.644903,-79.381836,The Fairmont Royal York,43.645449,-79.381508,Hotel
2,Central Toronto,43.644903,-79.381836,Pilot Coffee Roasters,43.645018,-79.380415,Coffee Shop
3,Central Toronto,43.644903,-79.381836,iQ Food Co,43.642851,-79.382081,Salad Place
4,Central Toronto,43.644903,-79.381836,Delta Hotels by Marriott Toronto,43.643069,-79.383865,Hotel
5,Central Toronto,43.644903,-79.381836,Real Sports Apparel,43.642860,-79.380184,Sporting Goods Shop
6,Central Toronto,43.644903,-79.381836,Balzac's Coffee,43.644373,-79.383065,Coffee Shop
7,Central Toronto,43.644903,-79.381836,Scotiabank Arena,43.643446,-79.379040,Basketball Stadium
8,Central Toronto,43.644903,-79.381836,Canoe,43.647452,-79.381320,Restaurant
9,Central Toronto,43.644903,-79.381836,Le Germain Hotel Toronto Maple Leaf Square,43.643125,-79.380918,Hotel


In [43]:
borough_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Central Toronto,100,100,100,100,100,100
Downtown Toronto,100,100,100,100,100,100
East Toronto,100,100,100,100,100,100
East York,8,8,8,8,8,8
Etobicoke,10,10,10,10,10,10
Mississauga,4,4,4,4,4,4
North York,52,52,52,52,52,52
Queen's Park,67,67,67,67,67,67
Scarborough,39,39,39,39,39,39


In [44]:
print('There are {} uniques categories.'.format(len(borough_venues['Venue Category'].unique())))

There are 119 uniques categories.


In [45]:
# one hot encoding
borough_onehot = pd.get_dummies(borough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
borough_onehot['Borough'] = borough_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [borough_onehot.columns[-1]] + list(borough_onehot.columns[:-1])
borough_onehot = borough_onehot[fixed_columns]

borough_onehot.head()

,Borough,Accessories Store,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,Central Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,Central Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Central Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Central Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Central Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
borough_grouped = borough_onehot.groupby('Borough').mean().reset_index()
borough_grouped

,Borough,Accessories Store,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,Central Toronto,0.00,0.020000,0.04,0.01,0.000000,0.000000,0.000,0.010000,0.000000,...,0.010000,0.00,0.000000,0.01,0.000000,0.000000,0.000000,0.01,0.0,0.000000
1,Downtown Toronto,0.01,0.030000,0.00,0.01,0.000000,0.010000,0.000,0.020000,0.000000,...,0.010000,0.02,0.010000,0.00,0.010000,0.000000,0.000000,0.00,0.0,0.000000
2,East Toronto,0.00,0.020000,0.04,0.01,0.000000,0.000000,0.000,0.010000,0.000000,...,0.010000,0.00,0.000000,0.01,0.000000,0.000000,0.000000,0.01,0.0,0.000000
3,East York,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.125,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000000
4,Etobicoke,0.00,0.100000,0.00,0.00,0.000000,0.000000,0.000,0.000000,0.100000,...,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000000
5,Mississauga,0.00,0.000000,0.25,0.00,0.000000,0.000000,0.000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000000
6,North York,0.00,0.019231,0.00,0.00,0.019231,0.019231,0.000,0.000000,0.019231,...,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.019231,0.00,0.0,0.019231
7,Queen's Park,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000,0.000000,0.000000,...,0.014925,0.00,0.000000,0.00,0.029851,0.000000,0.000000,0.00,0.0,0.000000
8,Scarborough,0.00,0.025641,0.00,0.00,0.000000,0.000000,0.000,0.025641,0.025641,...,0.025641,0.00,0.025641,0.00,0.000000,0.025641,0.000000,0.00,0.0,0.025641
9,West Toronto,0.00,0.020000,0.04,0.01,0.000000,0.000000,0.000,0.010000,0.000000,...,0.010000,0.00,0.000000,0.01,0.000000,0.000000,0.000000,0.01,0.0,0.000000


In [47]:
num_top_venues = 7

for hood in borough_grouped['Borough']:
    #print("----"+hood+"----")
    temp = borough_grouped[borough_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

                venue  freq
0         Coffee Shop  0.17
1                Café  0.06
2               Hotel  0.05
3       Deli / Bodega  0.05
4            Aquarium  0.04
5  Italian Restaurant  0.04
6          Restaurant  0.04


                 venue  freq
0       Clothing Store  0.08
1          Coffee Shop  0.05
2       Cosmetics Shop  0.04
3                 Café  0.04
4             Tea Room  0.03
5  American Restaurant  0.03
6           Restaurant  0.03


                venue  freq
0         Coffee Shop  0.17
1                Café  0.06
2               Hotel  0.05
3       Deli / Bodega  0.05
4            Aquarium  0.04
5  Italian Restaurant  0.04
6          Restaurant  0.04


                venue  freq
0         Coffee Shop  0.25
1  Italian Restaurant  0.12
2         Pizza Place  0.12
3      Sandwich Place  0.12
4  Athletics & Sports  0.12
5                Café  0.12
6      Farmers Market  0.12


                 venue  freq
0                Hotel   0.1
1                  Gym   0.1
2

In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [49]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
borough_venues_sorted = pd.DataFrame(columns=columns)
borough_venues_sorted['Borough'] = borough_grouped['Borough']

for ind in np.arange(borough_grouped.shape[0]):
    borough_venues_sorted.iloc[ind, 1:] = return_most_common_venues(borough_grouped.iloc[ind, :], num_top_venues)

borough_venues_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Central Toronto,Coffee Shop,Café,Deli / Bodega,Hotel,Aquarium,Restaurant,Italian Restaurant,Sports Bar,Sporting Goods Shop,Salad Place,Irish Pub,Concert Hall,Scenic Lookout,Seafood Restaurant,Gym
1,Downtown Toronto,Clothing Store,Coffee Shop,Cosmetics Shop,Café,American Restaurant,Italian Restaurant,Tea Room,Restaurant,Pizza Place,Sandwich Place,Theater,Breakfast Spot,Gastropub,Bubble Tea Shop,Bakery
2,East Toronto,Coffee Shop,Café,Deli / Bodega,Hotel,Aquarium,Restaurant,Italian Restaurant,Sports Bar,Sporting Goods Shop,Salad Place,Irish Pub,Concert Hall,Scenic Lookout,Seafood Restaurant,Gym
3,East York,Coffee Shop,Pizza Place,Sandwich Place,Italian Restaurant,Athletics & Sports,Farmers Market,Café,Electronics Store,Donut Shop,Discount Store,Department Store,Diner,Dessert Shop,Deli / Bodega,Cosmetics Shop
4,Etobicoke,Bank,American Restaurant,Hotel,Convenience Store,Gym,Grocery Store,Farmers Market,Coffee Shop,Clothing Store,Café,Discount Store,Electronics Store,Donut Shop,Wings Joint,Diner
5,Mississauga,Aquarium,Intersection,Pub,Bus Station,Wings Joint,Fast Food Restaurant,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Electronics Store,Falafel Restaurant
6,North York,Korean Restaurant,Ramen Restaurant,Coffee Shop,Restaurant,Sandwich Place,Pizza Place,Café,Japanese Restaurant,Dessert Shop,Burrito Place,Pool,Plaza,Pet Store,Movie Theater,Wings Joint
7,Queen's Park,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Ice Cream Shop,Vegetarian / Vegan Restaurant,Japanese Restaurant,Park,Indian Restaurant,Office,Restaurant,Bubble Tea Shop,Portuguese Restaurant,Burrito Place,Pharmacy
8,Scarborough,Clothing Store,Cosmetics Shop,Sporting Goods Shop,Sandwich Place,Coffee Shop,Tea Room,Wings Joint,Smoothie Shop,Shopping Mall,Fast Food Restaurant,Big Box Store,Pizza Place,Discount Store,Department Store,Men's Store
9,West Toronto,Coffee Shop,Café,Deli / Bodega,Hotel,Aquarium,Restaurant,Italian Restaurant,Sports Bar,Sporting Goods Shop,Salad Place,Irish Pub,Concert Hall,Scenic Lookout,Seafood Restaurant,Gym


In [88]:
# set number of clusters
kclusters = 5
borough_grouped_clustering = borough_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(borough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 0, 4, 1, 3, 1, 0, 1, 0], dtype=int32)

In [89]:
toronto_merged = df2
print toronto_merged
# add clustering labels
print kmeans.labels_
toronto_merged['Cluster Labels'] = kmeans.labels_
print toronto_merged

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(borough_venues_sorted.set_index('Borough'), on='Borough')

toronto_merged.head() # check the last columns!

             Borough   Latitude  Longitude  Cluster Labels
0    Central Toronto  43.644903 -79.381836               2
1   Downtown Toronto  43.655115 -79.380219               2
2       East Toronto  43.644903 -79.381836               2
3          East York  43.691339 -79.327821               0
4          Etobicoke  43.643556 -79.565633               2
5        Mississauga  43.667934 -79.587418               1
6         North York  43.770916 -79.412410               2
7       Queen's Park  43.659980 -79.390369               0
8        Scarborough  43.773077 -79.257774               2
9       West Toronto  43.644903 -79.381836               2
10              York  43.689619 -79.479188               3
[0 1 0 4 1 3 1 0 1 0 2]
             Borough   Latitude  Longitude  Cluster Labels
0    Central Toronto  43.644903 -79.381836               0
1   Downtown Toronto  43.655115 -79.380219               1
2       East Toronto  43.644903 -79.381836               0
3          East York  43.691339 

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Central Toronto,43.644903,-79.381836,0,Coffee Shop,Café,Deli / Bodega,Hotel,Aquarium,Restaurant,Italian Restaurant,Sports Bar,Sporting Goods Shop,Salad Place,Irish Pub,Concert Hall,Scenic Lookout,Seafood Restaurant,Gym
1,Downtown Toronto,43.655115,-79.380219,1,Clothing Store,Coffee Shop,Cosmetics Shop,Café,American Restaurant,Italian Restaurant,Tea Room,Restaurant,Pizza Place,Sandwich Place,Theater,Breakfast Spot,Gastropub,Bubble Tea Shop,Bakery
2,East Toronto,43.644903,-79.381836,0,Coffee Shop,Café,Deli / Bodega,Hotel,Aquarium,Restaurant,Italian Restaurant,Sports Bar,Sporting Goods Shop,Salad Place,Irish Pub,Concert Hall,Scenic Lookout,Seafood Restaurant,Gym
3,East York,43.691339,-79.327821,4,Coffee Shop,Pizza Place,Sandwich Place,Italian Restaurant,Athletics & Sports,Farmers Market,Café,Electronics Store,Donut Shop,Discount Store,Department Store,Diner,Dessert Shop,Deli / Bodega,Cosmetics Shop
4,Etobicoke,43.643556,-79.565633,1,Bank,American Restaurant,Hotel,Convenience Store,Gym,Grocery Store,Farmers Market,Coffee Shop,Clothing Store,Café,Discount Store,Electronics Store,Donut Shop,Wings Joint,Diner


In [90]:
# create map
map_clusters = folium.Map(location=[tor_lat, tor_long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 4. Cluster Neighborhoods

## 5. Examine Clusters

In [91]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,Borough,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Central Toronto,Café,Deli / Bodega,Hotel,Aquarium,Restaurant,Italian Restaurant,Sports Bar,Sporting Goods Shop,Salad Place,Irish Pub,Concert Hall,Scenic Lookout,Seafood Restaurant,Gym
2,East Toronto,Café,Deli / Bodega,Hotel,Aquarium,Restaurant,Italian Restaurant,Sports Bar,Sporting Goods Shop,Salad Place,Irish Pub,Concert Hall,Scenic Lookout,Seafood Restaurant,Gym
7,Queen's Park,Café,Italian Restaurant,Sandwich Place,Ice Cream Shop,Vegetarian / Vegan Restaurant,Japanese Restaurant,Park,Indian Restaurant,Office,Restaurant,Bubble Tea Shop,Portuguese Restaurant,Burrito Place,Pharmacy
9,West Toronto,Café,Deli / Bodega,Hotel,Aquarium,Restaurant,Italian Restaurant,Sports Bar,Sporting Goods Shop,Salad Place,Irish Pub,Concert Hall,Scenic Lookout,Seafood Restaurant,Gym


In [92]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,Borough,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
1,Downtown Toronto,Coffee Shop,Cosmetics Shop,Café,American Restaurant,Italian Restaurant,Tea Room,Restaurant,Pizza Place,Sandwich Place,Theater,Breakfast Spot,Gastropub,Bubble Tea Shop,Bakery
4,Etobicoke,American Restaurant,Hotel,Convenience Store,Gym,Grocery Store,Farmers Market,Coffee Shop,Clothing Store,Café,Discount Store,Electronics Store,Donut Shop,Wings Joint,Diner
6,North York,Ramen Restaurant,Coffee Shop,Restaurant,Sandwich Place,Pizza Place,Café,Japanese Restaurant,Dessert Shop,Burrito Place,Pool,Plaza,Pet Store,Movie Theater,Wings Joint
8,Scarborough,Cosmetics Shop,Sporting Goods Shop,Sandwich Place,Coffee Shop,Tea Room,Wings Joint,Smoothie Shop,Shopping Mall,Fast Food Restaurant,Big Box Store,Pizza Place,Discount Store,Department Store,Men's Store


In [93]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,Borough,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
10,York,Park,Grocery Store,Sandwich Place,Discount Store,Wings Joint,Farmers Market,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store


In [94]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,Borough,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
5,Mississauga,Intersection,Pub,Bus Station,Wings Joint,Fast Food Restaurant,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Electronics Store,Falafel Restaurant


In [95]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,Borough,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
3,East York,Pizza Place,Sandwich Place,Italian Restaurant,Athletics & Sports,Farmers Market,Café,Electronics Store,Donut Shop,Discount Store,Department Store,Diner,Dessert Shop,Deli / Bodega,Cosmetics Shop
